# **Space X  Falcon 9 First Stage Landing Prediction**


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


In this part, we will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


 ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import csv, sqlite3

In [2]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object.

In [3]:
response = requests.get(static_url)
html_content = response.text

if response.status_code == 200:
    print('Request Successfull')
else:
    print('Request Unsuccessfull')

Request Successfull


Create a `BeautifulSoup` object from the HTML `response`

In [4]:
soup = BeautifulSoup(html_content,'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 

In [5]:
page_title = soup.title
page_title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

Next, we want to collect all relevant column names from the HTML table header.

First, Let's try to find all tables on the wiki page first.

In [6]:
html_tables = soup.find_all('table')
len(html_tables)

25

As per research third table contains our desired record so we will collect it.

In [7]:
first_launch_table = html_tables[2]
#print(first_launch_table)

Next, we just need to iterate through the <th> elements and apply the provided extract_column_from_header() to extract column name one by one.
But before that, below are the required helper functions to process web scraped HTML table. Each function has a doc string which contains it's functionality.

In [8]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [9]:
th_element = first_launch_table.find_all('th')
th_element

[<th scope="col">Flight No.
 </th>,
 <th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
 </th>,
 <th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
 </th>,
 <th scope="col">Launch site
 </th>,
 <th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
 </th>,
 <th scope="col">Payload mass
 </th>,
 <th scope="col">Orbit
 </th>,
 <th scope="col">Customer
 </th>,
 <th scope="col">Launch<br/>outcome
 </th>,
 <th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">1
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">2
 </th>,
 <

In [10]:
column_names = []
for th in th_element:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [12]:
column_names.pop(1)
print(column_names)

['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Next, We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe.

In [13]:
launch_dict= dict.fromkeys(column_names)
launch_dict

{'Flight No.': None,
 'Launch site': None,
 'Payload': None,
 'Payload mass': None,
 'Orbit': None,
 'Customer': None,
 'Launch outcome': None}

In [14]:
# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the launch_dict with launch records extracted from table rows.

In [15]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number) 
            #print(flight_number)
           
            # Date value
            datatimelist=date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            #print(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            #print(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            #print(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            #print(orbit)
            
            # Customer
            if row[6].a:
                customer = row[6].a.string
                launch_dict['Customer'].append(customer)
            else:
                launch_dict['Customer'].append(None)  # or any other placeholder for missing data
            #print(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            #print(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)          

Now we can use this launch_dict to make a dataframe.

In [16]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,Not attempted\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0007.1,Not attempted\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...
116,117,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5B1051.10,Success,9 May 2021,06:42
117,118,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,F9 B5B1058.8,Success,15 May 2021,22:56
118,119,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5B1063.2,Success,26 May 2021,18:59
119,120,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success\n,F9 B5B1067.1,Success,3 June 2021,17:29


In [17]:
df.columns

Index(['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit',
       'Customer', 'Launch outcome', 'Version Booster', 'Booster landing',
       'Date', 'Time'],
      dtype='object')

In [18]:
column_rename = {
    'Flight No.': 'Flight No.',
    'Launch site': 'Launch_Site',
    'Payload': 'Payload',
    'Payload mass': 'PAYLOAD_MASS__KG_',
    'Orbit': 'Orbit',
    'Customer': 'Customer',
    'Launch outcome': 'Mission_Outcome',
    'Version Booster': 'Booster_Version',
    'Booster landing': 'Landing_Outcome',
    'Date': 'Date',
    'Time': 'Time (UTC)'
}

In [19]:
df = df.rename(columns=column_rename)
df.head()

,Flight No.,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Booster_Version,Landing_Outcome,Date,Time (UTC)
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,Not attempted\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0007.1,Not attempted\n,1 March 2013,15:10


In [20]:
new_sequence = ['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
                'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome', 'Landing_Outcome']
df = df[new_sequence]
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,4 June 2010,18:45,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,Failure
1,8 December 2010,15:43,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA,Success,Failure
2,22 May 2012,07:44,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA,Success,Not attempted\n
3,8 October 2012,00:35,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,No attempt
4,1 March 2013,15:10,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,Not attempted\n


## EDA With SQL

Let us first load the SQL extension and establish a connection with the database

In [21]:
%load_ext sql

In [22]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [23]:
%sql sqlite:///my_data1.db

In [24]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

121

The below code will remove the blank row from the table.

In [25]:
%%sql
create table SPACEXTABLE as select * from SPACEXTBL where Date is not null;

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [26]:
%sql SELECT * FROM SPACEXTBL LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
4 June 2010,18:45,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure
8 December 2010,15:43,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA,Success,Failure
22 May 2012,07:44,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA,Success,Not attempted
8 October 2012,00:35,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,No attempt
1 March 2013,15:10,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,Not attempted


Display the names of the unique launch sites  in the space mission

In [27]:
%%sql 
SELECT DISTINCT("Launch_Site") from SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS
VAFB
Cape Canaveral
KSC
CCSFS


Display 5 records where launch sites begin with the string 'CCA' 

In [28]:
%%sql 
SELECT * FROM SPACEXTBL WHERE "Launch_Site" LIKE 'CCA%' LIMIT 10;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
4 June 2010,18:45,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure
8 December 2010,15:43,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA,Success,Failure
22 May 2012,07:44,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA,Success,Not attempted
8 October 2012,00:35,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,No attempt
1 March 2013,15:10,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,Not attempted
3 December 2013,22:41,F9 v1.1,CCAFS,SES-8,"3,170 kg",GTO,SES,Success,Not attempted
6 January 2014,22:06,F9 v1.1,CCAFS,Thaicom 6,"3,325 kg",GTO,Thaicom,Success,Not attempted
8 January 2018,01:00,F9 B4,CCAFS,Zuma,C,LEO,Northrop Grumman,Success,Success
31 January 2018,21:25,F9 FTB1032.2,CCAFS,GovSat-1,"4,230 kg",GTO,SES,Success,Controlled
6 March 2018,05:33,F9 B4,CCAFS,Hispasat 30W-6,"6,092 kg",GTO,Hispasat,Success,No attempt


Display the total payload mass carried by boosters launched by NASA.

In [29]:
%%sql
SELECT DISTINCT("Orbit") FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Orbit
LEO
Polar orbit
GTO
HEO
Polar
SSO
MEO
Sub-orbital


In [30]:
%%sql
SELECT DISTINCT("Payload") FROM SPACEXTBL LIMIT 10;

 * sqlite:///my_data1.db
Done.


Payload
Dragon Spacecraft Qualification Unit
Dragon
SpaceX CRS-1
SpaceX CRS-2
CASSIOPE
SES-8
Thaicom 6
SpaceX CRS-3
Orbcomm-OG2
AsiaSat 8


In [31]:
%%sql
SELECT DISTINCT("Customer") FROM SPACEXTBL LIMIT 10;

 * sqlite:///my_data1.db
Done.


Customer
SpaceX
NASA
MDA
SES
Thaicom
Orbcomm
AsiaSat
USAF
ABS
None


In [32]:
%%sql 
SELECT DISTINCT("Landing_Outcome") from SPACEXTBL

 * sqlite:///my_data1.db
Done.


Landing_Outcome
Failure
Not attempted
No attempt
Uncontrolled
Not attempted
Controlled
Failure
Precluded
Success


In [33]:
%%sql 
SELECT MIN("Date") as FirstSuccessfulGroundPadLanding FROM SPACEXTBL WHERE Landing_Outcome == 'Success'

 * sqlite:///my_data1.db
Done.


FirstSuccessfulGroundPadLanding
1 May 2017


In [34]:
%%sql 
SELECT DISTINCT("Booster_Version") from SPACEXTBL 
WHERE Landing_Outcome == 'Success' AND (PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT♺
F9 FT
F9 FTB1031.2
F9 B5B1046.2
F9 B5B1047.2
F9 B5B1056.3
F9 B5B1058.2


In [35]:
%%sql 
SELECT COUNT(CASE WHEN Mission_Outcome LIKE 'Success%' THEN 1 END) AS Successful, 
COUNT(CASE WHEN Mission_Outcome LIKE 'Failure%' THEN 1 END) AS Failure FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Successful,Failure
120,1


In [36]:
%%sql
SELECT Booster_Version
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ IN (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5B1058.5


In [37]:
%%sql 
SELECT [Landing_Outcome], count(*) as count_outcomes FROM SPACEXTBL 
WHERE DATE between '04-06-2010' and '20-03-2017' group by [Landing_Outcome] order by count_outcomes DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,count_outcomes
Success,28
Failure,6
Not attempted,4
Controlled,3
Not attempted,2
Failure,1
